In [27]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Dataset locations downloader, samples from 2k locations 5 times each -> 10k image dataset

In [1]:
import matplotlib.pyplot as plt
import streetview
from skimage import io
import numpy as np
from IPython.display import Image
plt.rcParams['figure.figsize'] = [18, 9]
from IPython.display import clear_output

import csv
import requests
import json

In [2]:
#YOUR API KEY
key = "YOUR API KEY"

Information of 500 geoguessr rounds (each round is 5 locations)

In [7]:
with open('auto_geoguessr_3.csv', newline='') as csvfile:
    data = csv.reader(csvfile, delimiter=' ', quotechar='|')
    
    urls = [row[0].split(",")[0] for row in list(data)[1:]]
    
urls = [url for url in urls if url]    

In [8]:
len(urls)

459

In [32]:
ids = [url.split('/')[-1] for url in urls]

In [39]:
len(ids)

89

Download GeoGuessr API info

In [40]:
api_url = "https://geoguessr.com/api/v3/games/"
data = []

for id in ids:
    
  game_url = api_url+id
  response = requests.get(game_url)
  text = json.loads(response.text)['rounds']

  for round in text:

    if round["panoId"]:
        round["panoId_ascii"] = bytearray.fromhex(round["panoId"]).decode() 
    else:
        continue

    data.append(round)
    

In [41]:
print(json.dumps(data, indent=4))

[
    {
        "lat": 51.92945098876953,
        "lng": -3.827643871307373,
        "panoId": "4E4765396C757548507A53416F76734F663678745741",
        "heading": 0,
        "pitch": 0,
        "zoom": 0,
        "streakLocationCode": "gb",
        "panoId_ascii": "NGe9luuHPzSAovsOf6xtWA"
    },
    {
        "lat": 52.64149856567383,
        "lng": 1.2156277894973755,
        "panoId": "49687165534B30725564625038456434537138533777",
        "heading": 0,
        "pitch": 0,
        "zoom": 0,
        "streakLocationCode": "gb",
        "panoId_ascii": "IhqeSK0rUdbP8Ed4Sq8S7w"
    },
    {
        "lat": 50.3745231628418,
        "lng": -4.225609302520752,
        "panoId": "30393676707158546B3836396F367741366E51772D51",
        "heading": 0,
        "pitch": 0,
        "zoom": 0,
        "streakLocationCode": "gb",
        "panoId_ascii": "096vpqXTk869o6wA6nQw-Q"
    },
    {
        "lat": 53.75933074951172,
        "lng": -0.37022796273231506,
        "panoId": "534C7161613079772D756

In [42]:
with open('pano_info_5.json', 'w') as f:
  json.dump(data, f, ensure_ascii=False, indent = 4)

In [43]:
len(data) #number of geoguessr panos

12

In [44]:
import random

Location sampling

In [21]:
locations = []

index = 1
for round in data:
    
    lat,lng = round["lat"],round["lng"]
    
    try:
        req = streetview.download_panoids(lat, lng, key)
        
        panolat,panolng,panoid = req["location"]["lat"],req["location"]["lng"],req["pano_id"]
        locations.append((panolat,panolng,panoid))
    except:
        pass

    for i in range(4):
        s_lat,s_lng = lat + 0.5 * 2*(random.random() - 0.5), lng + 0.5 * 2*(random.random() - 0.5) #0.1° ~ 10km at equator
        
        try:
            req = streetview.download_panoids(s_lat, s_lng, key)
        
            panolat,panolng,panoid = req["location"]["lat"],req["location"]["lng"],req["pano_id"]
            locations.append((panolat,panolng,panoid))
        
        except:
            continue
    
    if index % 10 == 0:
        print(index,end= " ")
        
    index += 1

10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 222

Final dataset size

In [22]:
len(locations)

9278

In [23]:
print(json.dumps(locations, indent=4))

[
    [
        48.6684360093145,
        2.327627803565927,
        "WzIatXGoZKglEdpz7PV_Mg"
    ],
    [
        48.46312498441414,
        2.62568784950856,
        "VxP1CZNrla56lZAcaJDgcg"
    ],
    [
        48.76162310423111,
        2.043452162071988,
        "APAsxsUzofBlizmSZhdtdg"
    ],
    [
        49.00909897938914,
        2.700041127443947,
        "FDyTGfUFTQKHqwp177_yiQ"
    ],
    [
        49.02393542926135,
        2.15918534779344,
        "qS3WxE8_sm_Ys4K5FENm7A"
    ],
    [
        -26.32546877645128,
        27.64166608528284,
        "YokQt2chUmSfdC3RWEkuNg"
    ],
    [
        -25.94306937153055,
        27.6987214952977,
        "40z2tWDMqmVA6AUR3tC6Zg"
    ],
    [
        -26.71124605057467,
        27.65486345547429,
        "RoDB11M4XxJlm6W3z2qSlw"
    ],
    [
        -26.19003898701974,
        27.92857789901115,
        "dMDUrRENPiJeVBUSDydMoQ"
    ],
    [
        -26.75827514791183,
        27.29061347161738,
        "Wy0SaPn9AdDIOrWAftcWhw"
    

In [24]:
with open('pano_diverse_locations_5.json', 'w') as f:
  json.dump(locations, f, ensure_ascii=False, indent = 4)